<a href="https://colab.research.google.com/github/Mannafee/Kaggle-projects/blob/master/Categorical_feature_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
!pip install impyute
from impyute.imputation.cs import fast_knn


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
df_train= pd.read_csv('/content/drive/My Drive/Colab Notebooks/Categorical kaggle data files/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Categorical kaggle data files/test.csv')

# 1. **Data Import**

In [0]:
'''Variable Description'''
def description(df):
    #print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

In [0]:
df_train.drop("id",axis=1,inplace=True)
df_test.drop("id",axis=1,inplace=True)

In [0]:
description(df_train)

,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,id,int64,0,600000,0,1,2
1,bin_0,float64,17894,2,0,1,0
2,bin_1,float64,18003,2,0,1,1
3,bin_2,float64,17930,2,0,0,0
4,bin_3,object,18014,2,F,F,F
5,bin_4,object,18047,2,N,Y,N
6,nom_0,object,18252,3,Red,Red,Red
7,nom_1,object,18156,6,Trapezoid,Star,NaN
8,nom_2,object,18035,6,Hamster,Axolotl,Hamster
9,nom_3,object,18121,6,Russia,NaN,Canada


# 2. **Data Preprocessing**

### 2.1 Encoding categorical values to numerical

### NOTE: Label encoded the columns which are binary and ordinal columns leaving the Nan values for fast KNN prediction

In [0]:
lable_encod_col=['bin_3','bin_4','ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']
df_train[lable_encod_col] = df_train[lable_encod_col].apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))
df_test[lable_encod_col] = df_test[lable_encod_col].apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))


In [0]:
hot_encod_col=['nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9']
df_train = pd.concat([df_train,pd.get_dummies(df_train[hot_encod_col], prefix=hot_encod_col)],axis=1).drop(hot_encod_col,axis=1)
df_test = pd.concat([df_test,pd.get_dummies(df_test[hot_encod_col], prefix=hot_encod_col)],axis=1).drop(hot_encod_col,axis=1)

In [0]:
col=['bin_0','bin_1','bin_2','bin_3','bin_4','ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']
df_train[col]=fast_knn(df_train[col].values,k=3)
df_test[col]=fast_knn(df_test[col].values,k=3)

/usr/local/lib/python3.6/dist-packages/impyute/imputation/cs/fast_knn.py:113: RuntimeWarning: invalid value encountered in true_divide
  weights = distances/np.sum(distances)


In [0]:
# impute missing data
my_imputer = SimpleImputer(strategy='most_frequent')
df_train= pd.DataFrame(my_imputer.fit_transform(df_train),columns = df_train.columns)
df_test = pd.DataFrame(my_imputer.fit_transform(df_test),columns = df_train.columns)


In [0]:
Y_data = df_train["target"]
X_data = df_train[df_train.columns.difference(['target','bin_0','bin_1','bin_2','bin_3','bin_4','norm_0','norm_4','ord_0'])]
X_data = (X_data - X_data.mean())/X_data.std()
x_train, x_test, y_train, y_test =train_test_split(X_data,Y_data,train_size=0.8,test_size=0.2,random_state=1)

In [0]:
#random_forest = RandomForestClassifier(n_estimators=100)
#random_forest.fit(x_train, y_train)
#importances = pd.DataFrame({'feature':x_train.columns,'importance':np.round(random_forest.feature_importances_,3)})
#importances = importances.sort_values('importance',ascending=False).set_index('feature')
#importances.head(15)
#importances.plot.bar()

In [0]:
description(x_train)

In [0]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_train, y_train) * 100, 2)

In [0]:
acc_log

In [0]:
sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(x_train, y_train)
Y_pred = sgd.predict(x_test)
sgd.score(x_train, y_train)
acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)

In [0]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
Y_prediction = random_forest.predict(x_test)
random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train,y_train) * 100, 2)

In [0]:

acc_knn = 0

In [0]:

acc_gaussian = 0

In [0]:

acc_perceptron = 0

In [0]:

acc_linear_svc = 0

In [0]:
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(x_train, y_train)  
Y_pred = decision_tree.predict(x_test) 
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)

In [0]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

In [0]:
Xtest = df_test[df_test.columns.difference(['bin_0','bin_1','bin_2','bin_3','bin_4','norm_0','norm_4','ord_0'])]
Y_prediction_ = logreg.predict(Xtest)

In [0]:
df_submission=pd.read_csv('/kaggle/input/cat-in-the-dat-ii/test.csv')
df_submission.drop(df_submission.columns.difference(['id']), 1, inplace=True)
df_submission['target']=Y_prediction_
df_submission.head(10)

In [0]:
Y_prediction_

In [0]:
df_submission.to_csv('/content/drive/My Drive/Colab Notebooks/Categorical kaggle data files/gender_submission_2.csv', index = False)

In [0]:
#df_train = pd.concat([df_train,pd.get_dummies(df_train[hot_encod_col], prefix=hot_encod_col,dummy_na=True)],axis=1).drop(hot_encod_col,axis=1)